<img src="images\logo_datai.png" width="400" img style="float: right;"> 

https://www.unav.edu/web/instituto-de-ciencia-de-los-datos-e-inteligencia-artificial<br>
Author: Pablo Urruchi Mohino

## Download images from Google images

In [2]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    # Scrolls down to load more images
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))
    
    consent = wd.find_element(By.CSS_SELECTOR, 'button.VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-k8QpJ.VfPpkd-LgbsSe-OWXEXe-dgl2Hf.nCP5yc.AjY5Oe.DuMIQc.LQeN7.Nc7WLe')
    consent.click()
    
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        
        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, 'img.Q4LuWd')
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
        
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = thumbnail_results = wd.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_elements(By.CSS_SELECTOR, '.mye4qd')
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [9]:
def search_and_download(search_term: str, driver_path: str, target_path='./downloaded_images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) 
    # make the folder name inside images with the search string

    if not os.path.exists(target_folder):
        os.makedirs(target_folder) # make directory using the target path if it doesn't exist already

    with webdriver.Chrome() as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [10]:
DRIVER_PATH = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver'
search_term = 'cars dammage'
# num of images you can pass it from here  by default it's 10 if you are not passing
# number_images = 10
search_and_download(search_term=search_term
                    , number_images = 20
                    , driver_path=DRIVER_PATH) # method to download images

Found: 100 search results. Extracting links from 0:100
Found: 20 image links, done!
SUCCESS - saved https://www.theaa.com/~/media/the-aa/insurance/car-insurance/advice/damage-categories.jpg?rev=20b5ca5c01eb470fb1d3d01fbbbb7096&hash=4AB62BD9E62B92163D183239F47EFEB1 - as ./downloaded_images/cars_dammage
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQS2c0nW_qVTvT7RJ_RQ-l_J2yuo_uJAqPAlQ&usqp=CAU - as ./downloaded_images/cars_dammage
SUCCESS - saved https://di-uploads-pod3.dealerinspire.com/toyotaofclermont/uploads/2020/05/used-car-damage.jpg - as ./downloaded_images/cars_dammage
SUCCESS - saved https://www.mycarcredit.co.uk/wp-content/uploads/2019/11/car-crash-damage-categories.jpg - as ./downloaded_images/cars_dammage
SUCCESS - saved https://storage.googleapis.com/kx-live-ih/images/Collision_repair_shop-min.2e16d0ba.fill-800x450.jpg - as ./downloaded_images/cars_dammage
SUCCESS - saved https://www.shutterstock.com/image-photo/accident-car-260nw-682388278.jpg - as .

##### Question 1: what is "sleep" for?

##### Question 2: what is the purpose of {q} in the query

### Exploring what happens in variable "wb". The web browser

In [7]:
search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

# The following command opens the browser
wd = webdriver.Chrome()

In [6]:
# go to web page
wd.get(search_url.format(q='cars'))

In [7]:
# reject cookies
consent = wd.find_element(By.CSS_SELECTOR, 'button.VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-k8QpJ.VfPpkd-LgbsSe-OWXEXe-dgl2Hf.nCP5yc.AjY5Oe.DuMIQc.LQeN7.Nc7WLe')
consent.click()

In [8]:
# show all abailable images
wd.find_elements(By.CSS_SELECTOR, 'img.Q4LuWd')

[<selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="76f763d1-c140-4d52-8de0-e45df5e8059b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="ed74953b-a875-41c0-8e8a-882ec910d860")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="5e4da75e-e7cd-4f2c-b90d-6943ef60de99")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="2bcfe8fa-80b5-4e72-989a-5e3ff8e099f4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="1eaa3e19-914c-44a3-915b-8346657be383")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="3ae63595-b4c3-4a38-994f-fc8ff90dc0b9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="6401ddc6-3a48-47cb-b3fe-27

In [9]:
# Notice at this point we are not getting the actual png / jpg file but 
# a link to the image
x = wd.find_elements(By.CSS_SELECTOR, 'img.Q4LuWd')[0]
x

<selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="76f763d1-c140-4d52-8de0-e45df5e8059b")>

In [10]:
wd.find_elements_by_class_name("rg_i.Q4LuWd")[0]

<selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="76f763d1-c140-4d52-8de0-e45df5e8059b")>

In [11]:
wd.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')[0]

<selenium.webdriver.remote.webelement.WebElement (session="a7ae9fae59525a5e992ae3561feaaa31", element="76f763d1-c140-4d52-8de0-e45df5e8059b")>

<img src="images\picture_1.jpg" width="1000" img style="float: left;">

In [12]:
# This is why we perform a "click" action to the image. To open it and actually be able to get the image
x.click()

<img src="images\picture_2.jpg" width="1000" img style="float: left;">

In [13]:
wd

<selenium.webdriver.chrome.webdriver.WebDriver (session="a7ae9fae59525a5e992ae3561feaaa31")>

##### Question 3: find this button in your browser

##### Question 4: what you get from "fetch_image_urls"

##### Question 5: where is the actual request happening

##### Question 6: what happens if you close the prompted browser before finishing the execution?

__Comment:__ <br>
https://www.youtube.com/results?search_query=mannequin+challenge